In [11]:
import pandas as pd

df = pd.read_csv("../../data/data_raw_11:23.csv")

In [12]:
from data_preprocessing import preprocessing

In [13]:
preprocessing(df)

,PESSOA_PIPEDRIVE_id_person,PESSOA_PIPEDRIVE_id_gender,PESSOA_PIPEDRIVE_id_marrital_status,PESSOA_PIPEDRIVE_state,PESSOA_PIPEDRIVE_city,PESSOA_PIPEDRIVE_postal_code,PESSOA_PIPEDRIVE_contract_start_date,PESSOA_PIPEDRIVE_contract_end_date,PESSOA_PIPEDRIVE_Canal de Preferência,PESSOA_PIPEDRIVE_notes_count,...,WHOQOL_Psicológico_New,WHOQOL_Social_New,WHOQOL_Ambiental_New,COMUNICARE_Problemas Abertos Bool,TWILIO_Data Última Mensagens Inbound Recente,stay_time,last_stage_concluded,process_time,TWILIO_Data Última Mensagens Outbound Recente,TWILIO_Data Última Ligações Outbound Recente
25,483,64,80,Minas Gerais,Juiz de Fora,36021-600,2021-01-27,2023-08-26,0,3,...,4.0,4.0,4.5,0,False,211 days,Questionário,2022-04-19 15:19:40,True,True
29,654,64,82,Rio de Janeiro,Rio de Janeiro,20710-060,2021-04-25,2023-08-02,0,5,...,3.2,2.7,3.1,1,True,829 days,Questionário,2022-04-28 03:00:00,False,False
31,860,63,80,Paraná,Londrina,86020-410,2022-06-21,2023-09-20,239,3,...,3.0,4.0,4.0,1,True,456 days,Questionário,2022-06-27 15:03:30,True,False
37,961,63,80,Santa Catarina,Florianópolis,88080-080,2021-07-05,2021-10-01,0,1,...,3.0,3.0,3.0,0,False,87 days,Boas-vindas,2021-10-04 12:25:41,False,False
38,972,63,82,Minas Gerais,Uberlândia,38400-438,2021-06-28,2023-09-16,0,2,...,3.8,4.3,4.1,1,True,840 days,Questionário,2022-05-05 13:14:01,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,5944,64,80,Paraíba,João Pessoa,58052-310,2023-11-06,NaN,239,0,...,3.5,4.0,3.9,0,False,Em aberto,Boas-vindas,Em aberto,False,False
1197,5948,64,80,Bahia,Salvador,41830-120,2023-11-07,NaN,0,0,...,2.0,3.0,3.0,1,False,Em aberto,Primeira reunião,Em aberto,False,False
1198,5949,64,80,São Paulo,São Paulo,05014-001,2023-11-07,NaN,239,0,...,3.5,4.0,3.9,0,False,Em aberto,Boas-vindas,Em aberto,False,False
1200,5954,63,82,São Paulo,São Paulo,04513-100,2023-11-07,NaN,239,0,...,4.0,4.0,4.0,0,False,Em aberto,Boas-vindas,Em aberto,False,False
